

We're about to create and use our own MCP Server and MCP Client!



In [17]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown

load_dotenv(override=True)

True

In [18]:
from accounts import Account

In [19]:
account = Account.get("SS")
account

Account(name='ss', balance=9882.766, strategy='', holdings={'AMZN': 3}, transactions=[3 shares of AMZN at 39.078 each.], portfolio_value_time_series=[('2025-12-10 08:29:46', 10056.766), ('2025-12-10 08:30:15', 10071.766)])

In [20]:
account.buy_shares("AMZN", 3, "Because this bookstore website looks promising")

'Completed. Latest details:\n{"name": "ss", "balance": 9801.604, "strategy": "", "holdings": {"AMZN": 6}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 39.078, "timestamp": "2025-12-10 08:29:46", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 27.054, "timestamp": "2025-12-11 05:36:56", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-12-10 08:29:46", 10056.766], ["2025-12-10 08:30:15", 10071.766], ["2025-12-11 05:36:56", 10137.604]], "total_portfolio_value": 10137.604, "total_profit_loss": 137.60399999999936}'

In [21]:
account.report()

'{"name": "ss", "balance": 9801.604, "strategy": "", "holdings": {"AMZN": 6}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 39.078, "timestamp": "2025-12-10 08:29:46", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 27.054, "timestamp": "2025-12-11 05:36:56", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-12-10 08:29:46", 10056.766], ["2025-12-10 08:30:15", 10071.766], ["2025-12-11 05:36:56", 10137.604], ["2025-12-11 05:36:57", 10053.604]], "total_portfolio_value": 10053.604, "total_profit_loss": 53.60399999999936}'

In [22]:
account.list_transactions()

[{'symbol': 'AMZN',
  'quantity': 3,
  'price': 39.078,
  'timestamp': '2025-12-10 08:29:46',
  'rationale': 'Because this bookstore website looks promising'},
 {'symbol': 'AMZN',
  'quantity': 3,
  'price': 27.054,
  'timestamp': '2025-12-11 05:36:56',
  'rationale': 'Because this bookstore website looks promising'}]

### Now we write an MCP server and use it directly!

In [23]:
# Now let's use our accounts server as an MCP server

params = {"command": "uv", "args": ["run", "accounts_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()


In [24]:
mcp_tools

[Tool(name='get_balance', title=None, description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': 'get_balanceOutput', 'type': 'object'}, icons=None, annotations=None, meta=None),
 Tool(name='get_holdings', title=None, description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, outputSchema={'additionalProperties': {'type': 'integer'}, 'title': 'get_holdingsDictOutput', 'type': 'object'}, icons=None, annotations=None, meta=None),
 Tool(name='buy_shares', 

In [25]:
instructions = "You are able to manage an account for a client, and answer questions about the account."
request = "My name is SS and my account is under the name SS. What's my balance and my holdings?"
model = "gpt-4.1-mini"

In [26]:

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="account_manager", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("account_manager"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))


Your current cash balance is $9,801.60. You hold 6 shares of Amazon (AMZN) in your account. If you have any other questions or want to make changes, just let me know!

### Now let's build our own MCP Client

In [27]:
from accounts_client import get_accounts_tools_openai, read_accounts_resource, list_accounts_tools

mcp_tools = await list_accounts_tools()
print(mcp_tools)
openai_tools = await get_accounts_tools_openai()
print(openai_tools)

[Tool(name='get_balance', title=None, description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': 'get_balanceOutput', 'type': 'object'}, icons=None, annotations=None, meta=None), Tool(name='get_holdings', title=None, description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, outputSchema={'additionalProperties': {'type': 'integer'}, 'title': 'get_holdingsDictOutput', 'type': 'object'}, icons=None, annotations=None, meta=None), Tool(name='buy_shares', ti

In [30]:
request = "My name is SS and my account is under the name SS. What's my balance?"

with trace("account_mcp_client"):
    agent = Agent(name="account_manager", instructions=instructions, model=model, tools=openai_tools)
    result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Your account balance is $9,801.60. If you have any other questions or need assistance with your account, feel free to ask!

In [31]:
context = await read_accounts_resource("SS")
print(context)

{"name": "ss", "balance": 9801.604, "strategy": "", "holdings": {"AMZN": 6}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 39.078, "timestamp": "2025-12-10 08:29:46", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 27.054, "timestamp": "2025-12-11 05:36:56", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-12-10 08:29:46", 10056.766], ["2025-12-10 08:30:15", 10071.766], ["2025-12-11 05:36:56", 10137.604], ["2025-12-11 05:36:57", 10053.604], ["2025-12-11 05:38:15", 10203.604]], "total_portfolio_value": 10203.604, "total_profit_loss": 203.60399999999936}


In [33]:
from accounts import Account
Account.get("SS").report()

'{"name": "ss", "balance": 9801.604, "strategy": "", "holdings": {"AMZN": 6}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 39.078, "timestamp": "2025-12-10 08:29:46", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 27.054, "timestamp": "2025-12-11 05:36:56", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-12-10 08:29:46", 10056.766], ["2025-12-10 08:30:15", 10071.766], ["2025-12-11 05:36:56", 10137.604], ["2025-12-11 05:36:57", 10053.604], ["2025-12-11 05:38:15", 10203.604], ["2025-12-11 05:39:52", 10035.604]], "total_portfolio_value": 10035.604, "total_profit_loss": 35.60399999999936}'